In [2]:
# Load dependencies and set constants
from pyflink.datastream.functions import ProcessFunction
from pyflink.common.serialization import SimpleStringSchema, SerializationSchema
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment, TimeCharacteristic
from pyflink.datastream.connectors import FlinkKafkaConsumer, FlinkKafkaProducer
from matplotlib import pyplot as plt

import time
import sys

producer_props = {
    'bootstrap.servers': 'localhost:9092',
    'queue.buffering.max.messages': '1000000'
}

consumer_props = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'twitter-consumers',
}

KAFKA_TOPIC = "twitter-stream"
KAFKA_CONNECTOR_JAR = "file:///home/ubuntu/Stream-Analytics/flink/flink-sql-connector-kafka_2.11-1.12.2.jar"

# Number of tweets in the dataset (10 MB)
NUMBER_OF_TWEETS = 14485

# Number of times the dataset is produced by Kafka
NUMBER_OF_PRODUCTIONS = 10

In [7]:
latencies = []
records_received = 0

def collect_stats():
    plt.clf()
    plt.plot(latencies)
    plt.savefig('latency_plot.png')


class MyProcessFunction(ProcessFunction):
    def process_element(self, value, ctx: 'ProcessFunction.Context'):
        timestamp = ctx.timestamp()
        curr = time.time() * 1000
        latency = curr - timestamp
        
        #result = "Latency: {}   Record-Number: {}   Total Number of Records: {}".format(str(latency), str(len(latencies)), str(NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS))
#         result = str(latency + " | " + timestamp)
        result = str(latency)
        yield result
        
        # global might not be ideal here (parallelism possible?) but currently nothing else comes to mind
#         global records_received
#         records_received += 1
#         latencies.append(latency)
        
#         if records_received == NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS:
#             collect_stats()

env = StreamExecutionEnvironment.get_execution_environment()
'''
An execution environment defines a default parallelism for all operators, data sources,
and data sinks it executes
'''
#env.set_parallelism()
env.set_stream_time_characteristic(TimeCharacteristic.EventTime)

# Add the Kafka Connector Dependency
env.add_jars(KAFKA_CONNECTOR_JAR)
env.add_classpaths(KAFKA_CONNECTOR_JAR)

kafka_consumer = FlinkKafkaConsumer("twitter-stream", SimpleStringSchema(), consumer_props)
kafka_producer = FlinkKafkaProducer("twitter-stream-results", SimpleStringSchema(), producer_props)

stream = env.add_source(kafka_consumer)
stream \
    .process(MyProcessFunction(), output_type=Types.STRING()) \
    .print()
#     .add_sink(kafka_producer)

env.execute()

'''
Weird stuff:
    - Sometimes more records than emitted by Kafka (sometimes also less)
    - print output sometimes was "Latency: {}   Record-Number: {}   Total Number of Records: {}".format(str(latency), str(len(latencies)), str(NUMBER_OF_TWEETS * NUMBER_OF_PRODUCTIONS))
      and sometimes str(latency)!?!
    - what value for parallelism?
    
Notes:
    - Flink parallelism: https://stackoverflow.com/questions/61139187/what-is-the-difference-between-parallelism-and-parallel-computing-in-flink
'''




Py4JJavaError: An error occurred while calling o306.execute.
: org.apache.flink.runtime.client.JobExecutionException: Job execution failed.
	at org.apache.flink.runtime.jobmaster.JobResult.toJobExecutionResult(JobResult.java:144)
	at org.apache.flink.runtime.minicluster.MiniClusterJobClient.lambda$getJobExecutionResult$2(MiniClusterJobClient.java:117)
	at java.base/java.util.concurrent.CompletableFuture$UniApply.tryFire(CompletableFuture.java:642)
	at java.base/java.util.concurrent.CompletableFuture.postComplete(CompletableFuture.java:506)
	at java.base/java.util.concurrent.CompletableFuture.complete(CompletableFuture.java:2073)
	at org.apache.flink.runtime.rpc.akka.AkkaInvocationHandler.lambda$invokeRpc$0(AkkaInvocationHandler.java:237)
	at java.base/java.util.concurrent.CompletableFuture.uniWhenComplete(CompletableFuture.java:859)
	at java.base/java.util.concurrent.CompletableFuture$UniWhenComplete.tryFire(CompletableFuture.java:837)
	at java.base/java.util.concurrent.CompletableFutu

In [ ]:
# Load dependencies and set constants
import platform, socket, json, psutil, logging, multiprocessing

from time import time, perf_counter
from confluent_kafka import Consumer, KafkaError

# Create Kafka Consumer
consumer_config = {
    "bootstrap.servers": "localhost:9092",
    "group.id": "twitter-consumers",
    "client.id": "client-1",
    "enable.auto.commit": True,
    "session.timeout.ms": 6000,
    "default.topic.config": {"auto.offset.reset": "smallest"}
}
c = Consumer(consumer_config)

msg_counter = 0
total_messages = 9999999
c.subscribe(["twitter-stream-results"])

while msg_counter < total_messages:               
    msg = c.poll(0.5)

    if msg is None:
        continue
    elif not msg.error():
        msg_counter += 1
        print("Message {}: {}".format(msg_counter, msg.value()))
#         msg_counter += 1
        # Start the timer once the first message was received
#         if msg_counter == 1:
#             start = perf_counter()
#         latencies.append(latency)
    elif msg.error().code() == KafkaError._PARTITION_EOF:
        print("End of partition reached {}/{}".format(msg.topic(), msg.partition()))
        print("Messages consumed: {}".format(msg_counter))
    else:
        print("Error occured: {}".format(msg.error().str()))